In [1]:
import pandas as pd
import numpy as np
import torch

/home/austin/anaconda3/envs/Py9/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ercot_2022_actual_load_df = pd.read_csv('downloaded/20220101-20221212 ERCOT Actual Load.csv')
ercot_2022_actual_load_df.head(), ercot_2022_actual_load_df.tail()

(                   Date      Load
 0  1/1/2022 12:00:00 AM  38145.79
 1   1/1/2022 1:00:00 AM  37158.13
 2   1/1/2022 2:00:00 AM  35966.24
 3   1/1/2022 3:00:00 AM  35148.96
 4   1/1/2022 4:00:00 AM  34610.33,
                         Date      Load
 8299   12/12/2022 7:00:00 PM  46775.92
 8300   12/12/2022 8:00:00 PM  46217.27
 8301   12/12/2022 9:00:00 PM  44998.13
 8302  12/12/2022 10:00:00 PM  42774.80
 8303  12/12/2022 11:00:00 PM  40435.58)

In [21]:

def train_test_split(df, test_size=0.2):
    train_size = int(len(df) * (1 - test_size))
    train_set = df[:train_size]
    test_set = df[train_size:]
    return train_set, test_set

def train_test_split_by_date(df):
    train_set = df[df['Date'] < '2022-10-01']
    test_set = df[df['Date'] >= '2022-10-01']
    return train_set, test_set
    

train_set, test_set = train_test_split_by_date(ercot_2022_actual_load_df)
train_set.shape, test_set.shape

((3169, 2), (5135, 2))

In [22]:
class TimeseriesDataset(torch.utils.data.Dataset):   
    def __init__(self, X, y, seq_len=1):
        self.X = X
        self.y = X
        self.seq_len = seq_len

    def __len__(self):
        return self.X.__len__() 

    def __getitem__(self, index):
        #print(index)
        a, b = self.X[index:index+self.seq_len], self.y[index+self.seq_len]
        #print(a.shape, b.shape)
        return(a, b)

In [23]:
def df_to_tensor(df):
    X = torch.tensor(df['Load'].values).float()
    return X
#print(test_set.values[1663])
x_time_series = df_to_tensor(train_set)
y_time_series = df_to_tensor(test_set)
x_time_series.shape, y_time_series.shape

(torch.Size([3169]), torch.Size([5135]))

In [24]:
seq_len = 73
batch_size = seq_len
train_dataset = TimeseriesDataset(x_time_series, y_time_series, seq_len=seq_len)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size, shuffle = False)


# for nth_batch, (batch, _) in enumerate(train_dataset):
#     print(f'Batch {nth_batch}:')
#     print(f'batch shape: {batch.shape}')
#     print(f'batch: {batch}')


In [25]:
import torch
import torch.nn as nn
dims = seq_len

class Transformer(nn.Module):
  def __init__(self, input_dim, output_dim, hidden_dim, num_layers, num_heads):
    super(Transformer, self).__init__()

    self.encoder = nn.TransformerEncoder(
      nn.TransformerEncoderLayer(input_dim, num_heads, hidden_dim),
      num_layers
    )

    self.decoder = nn.TransformerDecoder(
      nn.TransformerDecoderLayer(output_dim, num_heads, hidden_dim),
      num_layers
    )

    self.output_layer = nn.Linear(hidden_dim, output_dim)

  def forward(self, src, trg):
    context = self.encoder(src)
    output = self.decoder(trg, context)
    return self.output_layer(output)

Transformer = Transformer(input_dim=dims, output_dim=dims, hidden_dim=seq_len, num_layers=4, num_heads=seq_len)

In [26]:
from tqdm import tqdm
import os
def train_transformer(model, train_loader, epochs):
  criterion = nn.MSELoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

  for epoch in range(epochs):
    for batch, _ in tqdm(train_loader):
      #print(batch.shape)
      #batch2 = batch.transpose(-2, -1)

      #print(batch.shape)
      output = model(batch, batch)
      loss = criterion(output, batch)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item():.4f}')


if os.path.exists('transformer_model.pt'):
  Transformer = torch.load('transformer_model.pt')
  
else:    
    train_transformer(Transformer, train_loader, epochs=100)
    torch.save(Transformer.state_dict(), 'transformer_model.pt')

 95%|█████████▌| 42/44 [00:01<00:00, 33.74it/s]


IndexError: index 3169 is out of bounds for dimension 0 with size 3169

In [ ]:
test_set.head()

,Date,Load
6643,10/4/2022 8:00:00 PM,50943.27
6644,10/4/2022 9:00:00 PM,48196.03
6645,10/4/2022 10:00:00 PM,44963.52
6646,10/4/2022 11:00:00 PM,41779.37
6647,10/5/2022 12:00:00 AM,38623.86


In [ ]:
def test_transformer(model, test_loader):
  with torch.no_grad():
    for batch, _ in test_loader:
      #batch = batch.permute(1, 0, 2)
      output = model(batch)
      print(f'Input: {batch}')
      print(f'Output: {output}')
test_transformer(Transformer, train_loader)

TypeError: forward() missing 1 required positional argument: 'trg'

In [ ]:
import plotly.express as px 
def plot_predictions(model, test_loader):
    """Using Plotly, plot predictions of the model on the test set against the actual values for a single batch. The y-axis is 'Load'. The x-axis is the index of the test set."""
    with torch.no_grad():
        for batch, _ in test_loader:
            output = model(batch, batch)
            output = output.flatten()
            batch = batch.flatten()
            df = pd.DataFrame({'Load': output, 'Predicted Load': batch})
            df = df.reset_index()
            df = df.melt(id_vars='index', value_vars=['Load', 'Predicted Load'])
            fig = px.line(df, x='index', y='value', color='variable')
            fig.show()
            break
plot_predictions(Transformer, train_loader)


NameError: name 'Transformer' is not defined

In [ ]:
def summarize_transformer(model):
  print(model)
summarize_transformer(Transformer)

Transformer(
  (encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=73, out_features=73, bias=True)
        )
        (linear1): Linear(in_features=73, out_features=73, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=73, out_features=73, bias=True)
        (norm1): LayerNorm((73,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((73,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
      (1): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=73, out_features=73, bias=True)
        )
        (linear1): Linear(in_features=73, out_features=73, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
  